# Importation des Package

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import bs4
import re
import copy
import collections as co
import plotly
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np


# Webscrapping du top 10 netflix journalier

#### Fonction qui récupère le dataframe quand on lui donne la date sous forme string "aaaa/mm/jj", url est un argument conditionnel.

In [46]:
def recup_top_netflix_gen(date,url="https://www.the-numbers.com/home-market/netflix-daily-chart/"):
    
    url_enrichi = url+date
    from urllib.request import Request, urlopen
    req_text = urlopen(Request(url_enrichi, headers={'User-Agent': 'Mozilla/5.0'})).read()
    page = bs4.BeautifulSoup(req_text, "lxml")
    #print (page)

    """On récupère les données------------------"""
    #tableau=page.find("table",{'class' : 'chart full-width'})
    tableau=page.find("div",{'id':'page_filling_chart'})
    #print(tableau)
    #table_body=tableau.find('tbody')
    #print(table_body)
    rows = tableau.find_all('tr')
    #print (rows)

  
    """On récupére tous les élements correspondant au titre et à la note et On crée un dictionnaire dans lequel on met le contenu des lignes nettoyées
    L'index est le titre, la valeur la note.
    --------------"""
    new_rows=[]#on crée une liste vide dans laquelle on va mettre le contenu des lignes
    new_cols = 10*[[]]
    dico_top_netflix = dict()
    i = 0
    for row in rows:
        cols = row.find_all('td') # c'est une liste
        cols = [ele.text.strip() for ele in cols]
        #print (cols)
        if len(cols)> 0:
            dico_top_netflix[cols[0]] = cols[1:]
    #print(dico_top_netflix)    

    """"""

    """On crée ENFIN le dataframe"""
    df_top_netflix_gen = pd.DataFrame.from_dict(dico_top_netflix,orient='index')

    """On rajoute une index différente du titre"""
    for row in rows:
        cols = row.find_all('th')
        #print(cols)
        if len(cols) > 0 : 
            cols = [ele.get_text(separator=' ').strip().title() for ele in cols]
            columns_title = cols
    #print (columns_title)
    

    """On change le nom des colonnes"""
    df_top_netflix_gen.columns=columns_title[1:]
    
    """On vire les colonnes qui ne nous intéresse pas"""
    del(df_top_netflix_gen['Yd'])
    del(df_top_netflix_gen['Lw'])
    del(df_top_netflix_gen['Watch Now'])
    
    """On rajoute une colonne 'Measure Date' """
    df_top_netflix_gen['Measure Date']= [date]*df_top_netflix_gen.shape[0]
    #print ("Graph au "+ date)
    return (df_top_netflix_gen)

## Concaténer les tableaux

Liste de toutes les dates valides

In [31]:
liste_dates_valides=[]
for m in range (3,13):
    m=str(m)
    if len(m)==1:
        m = "0"+m
    for d in range (1,32):
        d=str(d)
        if len(d)==1:
            d="0"+d
        date = "2020/"+m+"/"+d
        liste_dates_valides.append(date)

#On supprime les dates sans valeurs
for i in ['2020/03/01', '2020/03/02', '2020/03/03', '2020/03/04', '2020/03/05', '2020/03/06', '2020/03/07', '2020/03/08', 
          '2020/03/09', '2020/03/10', '2020/03/11', '2020/03/12', '2020/03/13', '2020/03/14', '2020/03/15', '2020/03/16', 
          '2020/03/17', '2020/03/18', '2020/03/19', '2020/03/20', '2020/03/21', '2020/03/22', '2020/03/23', '2020/03/28', 
          '2020/03/29', '2020/04/04', '2020/04/05', '2020/04/11', '2020/04/12', '2020/04/18', '2020/04/19', '2020/04/25', 
          '2020/04/26', '2020/08/09', '2020/08/30']:
    liste_dates_valides.remove(i)


In [48]:
df_06_05=recup_top_netflix_gen("2020/06/05")
df_06_06=recup_top_netflix_gen("2020/06/06")
DF=pd.concat([df_06_05, df_06_06])
DF.sort_values('Measure Date',ascending=False)
